# FRAGSTATS metrics comparison

In [ ]:
import numpy as np
import pandas as pd

import pylandstats as pls

The aim of this notebook is to compare the computed values for the landscape metrics of PyLandStats with those of FRAGSTATS (v4.2). Let us first set the relative tolerance to 0.001, which means that we accept a relative difference of 0.1% between the values computed with PyLandStats and FRAGSTATS. We will also define some utilities.

In [ ]:
# definitions

fragstats_abbrev_dict = pls.settings.fragstats_abbrev_dict
tol = 1e-3

In [ ]:
# utils


def read_fragstats_csv(csv_filepath):
    # `na_values` is required because of the leading whitespaces that FRAGSTATS
    # leaves when saving CSV files
    fragstats_df = pd.read_csv(csv_filepath, na_values=[" N/A"])
    fragstats_df.columns = fragstats_df.columns.str.strip()
    try:
        fragstats_df["TYPE"] = (
            fragstats_df["TYPE"].str.strip().str.replace("cls_", "").astype(int)
        )
    except KeyError:
        pass

    return fragstats_df

Below is the reference basename for the extract of the Canton of Vaud (Switerland) derived from the [CORINE Land Cover dataset](https://land.copernicus.eu/pan-european/corine-land-cover) of the year 2000.

In [ ]:
basename = "vaud_g100_clc00_V18_5"

We will instantiate a PyLandStats landscape with the respective raster file:

In [ ]:
ls = pls.Landscape("../data/raw/clc/{}.tif".format(basename), res=(100, 100))

Now we will use pandas to load the `.patch`, `.class` and `.landscape` files (which are dumps of the metrics computed in FRAGSTATS), and then compare the computed values with those of PyLandStats. If the value for any metric differs more than the relative tolerance defined above, a `RuntimeError` will be raised.

In [ ]:
patch_df = read_fragstats_csv("../data/raw/fragstats/{}.patch".format(basename))

In [ ]:
for patch_metric in pls.Landscape.PATCH_METRICS:
    fragstats_abbrev = fragstats_abbrev_dict[patch_metric]
    for class_val in ls.classes:
        fragstats_ser = patch_df[fragstats_abbrev][patch_df["TYPE"] == class_val]
        pls_ser = getattr(ls, patch_metric)(class_val=class_val)
        if not np.allclose(fragstats_ser, pls_ser, tol, equal_nan=True):
            raise RuntimeError(patch_metric, class_val, fragstats_ser, pls_ser)
    print("{}: OK".format(patch_metric))

area: OK
perimeter: OK
perimeter_area_ratio: OK
shape_index: OK
fractal_dimension: OK
euclidean_nearest_neighbor: OK


In [ ]:
class_df = read_fragstats_csv("../data/raw/fragstats/{}.class".format(basename))

In [ ]:
for class_metric in pls.Landscape.CLASS_METRICS:
    if class_metric == "total_area":
        fragstats_abbrev = "CA"
    else:
        fragstats_abbrev = fragstats_abbrev_dict[class_metric]
    if class_metric not in pls.Landscape.DISTR_METRICS:
        for class_val in ls.classes:
            fragstats_val = class_df[fragstats_abbrev][
                class_df["TYPE"] == class_val
            ].iloc[0]
            pls_val = getattr(ls, class_metric)(class_val=class_val)
            if not (
                np.isclose(fragstats_val, pls_val, tol)
                or np.isclose(fragstats_val, pls_val, atol=tol)
            ):
                raise RuntimeError(
                    "{} (class {}): fragstats {}, pylandstats {}".format(
                        class_metric, class_val, fragstats_val, pls_val
                    )
                )

            print("{} (class {}): OK".format(class_metric, class_val))

total_area (class 1): OK
total_area (class 2): OK
proportion_of_landscape (class 1): OK
proportion_of_landscape (class 2): OK
number_of_patches (class 1): OK
number_of_patches (class 2): OK
patch_density (class 1): OK
patch_density (class 2): OK
largest_patch_index (class 1): OK
largest_patch_index (class 2): OK
total_edge (class 1): OK
total_edge (class 2): OK
edge_density (class 1): OK
edge_density (class 2): OK
landscape_shape_index (class 1): OK
landscape_shape_index (class 2): OK
effective_mesh_size (class 1): OK
effective_mesh_size (class 2): OK


In [ ]:
landscape_df = read_fragstats_csv("../data/raw/fragstats/{}.land".format(basename))

In [ ]:
for landscape_metric in pls.Landscape.LANDSCAPE_METRICS:
    fragstats_abbrev = fragstats_abbrev_dict[landscape_metric]
    # contagion is treated differently (see below)
    if (
        landscape_metric not in pls.Landscape.DISTR_METRICS
        or landscape_metric == "contagion"
    ):
        fragstats_val = landscape_df[fragstats_abbrev].iloc[0]
        pls_val = getattr(ls, landscape_metric)()
        if not np.isclose(fragstats_val, pls_val, rtol=0.01):
            raise RuntimeError(
                "{}: fragstats {}, pylandstats {}".format(
                    landscape_metric, fragstats_val, pls_val
                )
            )

        print("{}: OK".format(landscape_metric))

# Treat contagion differently: here we use a relative tolerance of .01 (1%
# relative error) because the computed contagion might have greater divergence
# with FRAGSTATS
landscape_metric = "contagion"
fragstats_abbrev = fragstats_abbrev_dict[landscape_metric]
fragstats_val = landscape_df[fragstats_abbrev].iloc[0]
pls_val = getattr(ls, landscape_metric)()

if not np.isclose(fragstats_val, pls_val, rtol=0.01):
    raise RuntimeError(
        "{}: fragstats {}, pylandstats {}".format(
            landscape_metric, fragstats_val, pls_val
        )
    )
print(
    "{}: OK (fragstats: {}, pylandstats: {})".format(
        landscape_metric, fragstats_val, pls_val
    )
)

total_area: OK
number_of_patches: OK
patch_density: OK
largest_patch_index: OK
total_edge: OK
edge_density: OK
landscape_shape_index: OK
effective_mesh_size: OK
contagion: OK
shannon_diversity_index: OK
contagion: OK (fragstats: 73.7483, pylandstats: 73.40177013078969)
